In [86]:
import os
#import ipdb

import pandas as pd
import matplotlib as mpl
from matplotlib import lines
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

EXP_NAME = "c10"
headers = ['time','start_transfer', 'latency','gateway','status']
def read_client():
    INPUT_DIR = os.path.join( os.getcwd(), EXP_NAME)
    data = pd.DataFrame()
    dirs = [f for f in os.listdir(INPUT_DIR) if os.path.isdir(os.path.join(INPUT_DIR))]    
    for client in dirs:
        client_data = pd.read_csv(os.path.join(INPUT_DIR, client), sep=',')
        client_data.columns= headers
        client_data = client_data.set_index(pd.DatetimeIndex(client_data.time))
        client_data = client_data.query('status == 200')
        data = pd.concat([data, client_data])
    return data
    
def getECDF(df):
    """Helper function that caclulates the ECDF of a dataframe"""
    df = df.value_counts()
    ecdf = df.sort_index().cumsum()*1./df.sum()    
    return ecdf

In [87]:
data = read_client()
std_data = pd.DataFrame()

std_data = data['latency'].sort_index().rolling('10T').std().to_frame()
mean_data = data['latency'].sort_index().rolling('1T').mean().to_frame()
general  = pd.merge(std_data, mean_data, how='inner', left_index=True, right_index=True)
general.columns = ['std','mean']
std_data.to_csv('std_dev/general_std.csv')

for name, group in data.groupby(data['gateway']):
    group_std = pd.DataFrame()
    group_std = group['latency'].rolling('10T').std().to_frame()
    group_mean = group['latency'].rolling('1T').mean().to_frame()
    group_data = pd.merge(group_std, group_mean, how='inner', left_index=True, right_index=True)
    group_data.columns = ['std', 'mean']
    merged = pd.merge(group_data, general, how='inner', left_index=True, right_index=True)
    merged.columns = ['std', 'latency', 'all_std','mean_latency']
    merged.to_csv('std_dev/'+name+'.csv')
    


In [94]:
import os
#import ipdb

import pandas as pd
import matplotlib as mpl
from matplotlib import lines
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

EXP_NAME = "std_dev"
# Read all standard deviation files of each gateway node and count what category nodes are at each time period
def read_std():
    INPUT_DIR = os.path.join( os.getcwd(), EXP_NAME)
    data = pd.DataFrame()
    dirs = [f for f in os.listdir(INPUT_DIR) if os.path.isdir(os.path.join(INPUT_DIR))]    
    for client in dirs:
        client_data = pd.read_csv(os.path.join(INPUT_DIR, client), sep=',')
        client_data = client_data.set_index(pd.DatetimeIndex(client_data.time))
        client_data = client_data.head(1000)
        print(client)
        for index, row in client_data.iterrows():
            if row['mean_latency'] is None:
                continue
            if row['latency'] < row['mean_latency']:
                if row['std'] < row['all_std']:
                    client_data.loc[index, 'category'] = 'Good'
                else:
                    client_data.loc[index, 'category'] = 'Inconsistent'
            else:
                if row['std'] < row['all_std']:
                    client_data.loc[index, 'category'] = 'Inconsistent'
                else:
                    client_data.loc[index, 'category'] = 'Bad'
        
        data = pd.concat([data, client_data])
    return data
category_data = read_std()

10.140.93.35.csv
proxy.cecilia.guifi.net.csv
proxy.sbll.guifi.net.csv
10.139.37.194.csv
proxy.elpipa.guifi.net.csv
proxy.muntanyola.guifi.net.csv
proxy.malla.guifi.net.csv
10.139.40.122.csv
10.138.57.2.csv
proxy.olost.guifi.net.csv
10.138.120.66.csv
10.228.192.210.csv
10.228.193.210.csv
10.139.94.98.csv
proxy.smc.guifi.net.csv
10.138.25.67.csv
proxy.perafita.guifi.net.csv
10.139.40.85.csv
10.139.17.4.csv
10.138.85.130.csv


In [ ]:
for name, group in category_data.groupby([category_data.index, category_data['category']]):
    print(name, len(group))